In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

import io
import pickle
import numpy as np
import math
import panel as pn
from PIL import Image
import holoviews as hv
import altair as alt
from sklearn.decomposition import PCA
alt.data_transformers.disable_max_rows()
hv.extension("plotly")
pn.extension("plotly")
pn.config.theme = 'dark'
hv.renderer('plotly').theme = 'dark'
from torchvision import datasets
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from functools import partial, reduce

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
import torch
import functools
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import numpy as np
from einops import rearrange
from torch.optim.lr_scheduler import LambdaLR
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch.nn.functional as F
import gc
from tqdm import tqdm, trange
import lpips
import lpips
import os
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset
from torchvision.transforms.v2 import PILToTensor,Compose
import torchvision
import sys

sys.path.append(os.path.abspath(os.path.join('..')))
from src.models.unified_classifier import ClassifierModelPipeline, evaluate_test_set, evaluate_unified_classifier_model, predictDenoise, predictCLS, ClassificationHeadDummy
from src.md import MDDataset, generate_phases_for_dense_validation, MDDenseSet, MDLoadable, MDDense, MDDensePhaseData
from src.models.denoiser import get_forward_diffusion_params, forward_add_noise
from src.preprocessing import ChromoDataContainer
from src.models.conv_classifier import ConvChromoClassifier, train_conv_classifier_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# TRANING OF CONV CLASSIFIER MODEL

# creating val datasets for real data
chrom_data = ChromoDataContainer.load_from_pkl()

# TODO: Split MD and DenseD to separatte notebooks
# 1) MD generates source for Dense MD
# 2) DenseD generates Both Dense MD and Dense RD
distiled_real_mu = generate_phases_for_dense_validation(chrom_data.gausians, do_real_mu=True)
distiled_real_lowess = generate_phases_for_dense_validation(chrom_data.gausians, do_real_mu=False)
distiled_mock_mu = MDLoadable.load(
    config_name="dataset_128_128_784_classifier_control_set_mu",
    config_folder="../configs/MDDense",
    dataset_folder="../data/MDDense",
    dataset_cls=MDDenseSet)
distiled_mock_lowess = MDLoadable.load(
    config_name="dataset_128_128_784_classifier_control_set_lowess",
    config_folder="../configs/MDDense",
    dataset_folder="../data/MDDense",
    dataset_cls=MDDenseSet)

# load dataset for training

dataset = MDDataset.load(
    config_name="dataset_28_28_784_classifier",
    dataset_cls=MDDataset)

model = ConvChromoClassifier(seq_len=784, class_len=784).to(device)

train_conv_classifier_model(
    dataset=dataset,
    model=model,
    run_name = 'conv_classifier',
    n_epochs = 512,
    batch_size = 768*4,
    
    timestep = 1, # fixed timestep for the base denoiser model
    label_num = 5, # fixed label num for the base denoiser model
    label_count = 10, # fixed label count for the base denoiser model
    
    lr=10e-3,
    validation_portion=0.1,
    validation_per_epoch = 10,
    validation_samples = 768*4,
    
    distilled_val_sets = (
        distiled_real_mu, 
        distiled_real_lowess, 
        distiled_mock_mu, 
        distiled_mock_lowess), # TODO: refactor this to have better generalization
    
    checkpoints_folder='../checkpoints/classifier'
)